In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [22]:
vax_path = "/Users/jeandavidt/Library/CloudStorage/OneDrive-UniversitéLaval/Université/Doctorat/COVID/Latest Data/Input/MI/Cumulative vaccinations area.xlsx"

df = pd.read_excel(vax_path, header=1)
df["VAX_WEEK"] = df["VAX_WEEK"].ffill()
df["VAX_WEEK"] = pd.to_datetime(df["VAX_WEEK"])


vals = []
for idx, row in df.iterrows():
    for col in df.columns[4:]:
        if not np.isnan(row[col]):
            vals.append(row[col])
            continue
df = df.drop(columns=df.columns[4:])
df["percentages"] = vals

df = df.drop(columns=["Vax week whitespace buffer", "Age"])

df = df.rename(columns={"Unnamed: 3": "variable"})
df = df[df["variable"] != "Up to date"]
df["variable"] = df["variable"].map(
    {
        "At least one dose": "pctVaccineDose1",
        "Completed vaccine series": "pctVaccineDose2",
    }
)

df.head()


,VAX_WEEK,variable,percentages
1,2020-12-06,pctVaccineDose2,0.000031
2,2020-12-06,pctVaccineDose1,0.000068
4,2020-12-13,pctVaccineDose2,0.000033
5,2020-12-13,pctVaccineDose1,0.001182
7,2020-12-20,pctVaccineDose2,0.000037


In [7]:
files = {
    "hosp": "/Users/jeandavidt/Library/CloudStorage/OneDrive-UniversitéLaval/Université/Doctorat/COVID/Latest Data/Input/MI/hcounty.csv",
    "deaths": "/Users/jeandavidt/Library/CloudStorage/OneDrive-UniversitéLaval/Université/Doctorat/COVID/Latest Data/Input/MI/dcounty.csv",
    "wwtp": "/Users/jeandavidt/Library/CloudStorage/OneDrive-UniversitéLaval/Université/Doctorat/COVID/Latest Data/Input/MI/Metro Plant data.xlsx",
    "pct_pos": "/Users/jeandavidt/Library/CloudStorage/OneDrive-UniversitéLaval/Université/Doctorat/COVID/Latest Data/Input/MI/ww_MN metro percent positive.xlsx"
}
COUNTIES = [
    "Anoka County",
    "Washington County",
    "Dakota County",
    "Ramsey County",
    "Hennepin County",
    "Carver County",
    "Scott County"
]


In [35]:
df = pd.read_csv(files["hosp"])
print(len(df), " rows")
df.head()

25174  rows


,adm_date_mmwr,county,case_count,mmwr_startdate,mmwr_enddate,pop_size,outcome,rate
0,202004,Hennepin County,1,01/19/2020,01/25/2020,1245837,hospitalization,0.1
1,202004,Total,1,01/19/2020,01/25/2020,5563378,hospitalization,0.0
2,202009,Hennepin County,1,02/23/2020,02/29/2020,1245837,hospitalization,0.1
3,202009,Total,1,02/23/2020,02/29/2020,5563378,hospitalization,0.0
4,202010,Aitkin County,0,03/01/2020,03/07/2020,15834,hospitalization,0.0


In [36]:
df["mmwr_startdate"] = pd.to_datetime(df["mmwr_startdate"], format="%m/%d/%Y")
df["mmwr_enddate"] = pd.to_datetime(df["mmwr_enddate"], format="%m/%d/%Y")

In [37]:
df2 = df.groupby("mmwr_startdate").sum(numeric_only=True)

df2 = df2.sort_index()

In [38]:
ser1 = df2["case_count"]

ser2 = ser1.asfreq("24H").interpolate() / 7
ser2.name = "CPHD_hospcen_report_value"

px.scatter(ser2)


In [39]:
df = pd.DataFrame({"original":pd.Series(np.ones(100))})
df["rolling_sum"] = df["original"].rolling(4, center=False, min_periods=1).sum()
df

,original,rolling_sum
0,1.0,1.0
1,1.0,2.0
2,1.0,3.0
3,1.0,4.0
4,1.0,4.0
...,...,...
95,1.0,4.0
96,1.0,4.0
97,1.0,4.0
98,1.0,4.0


In [34]:
# building the active cases time series
ser3 = ser2.rolling(14, center=False, min_periods=1).sum()
px.scatter(pd.DataFrame({"new cases": ser2, "active cases": ser3}))


In [14]:
resampled = ser2.resample("1W-SUN", convention="start").sum()
resampled.name = "CPHD"

px.scatter(pd.DataFrame({"resampled": resampled, "original": df2["case_count"]}))

In [4]:

df = df.loc[df["county"].isin(COUNTIES)]
print(len(df), " rows")


NameError: name 'df' is not defined

In [9]:
df = pd.read_excel(files["wwtp"])
df.head()


,Date,N1 value (copies/L),N1 Stnd error (copies/L),N2 value (copies/L),N2 Stnd error (copies/L),Flow (million gallons/d),Influent Suspended Solids (mg/l),Influent Temperature (C),Influent CBOD-5day Liquids (mg/l),Influent COD (mg/l),Influent Ammonia Nitrogen (mgN/l),Influent Total Kjeldahl Nitrogen (mgTKN/l),Influent Total Phosphorus (mgP/l,Influent Maximum pH (su),Influent Minimum pH (su)
0,2020-11-01,504561.2625,nd,686687.625,nd,156.820770,146,18.0,260,438,28.900000,46,4.76,7.15,6.69
1,2020-11-02,583589.240625,nd,853229.98125,nd,163.118896,230,18.0,300,572,29.799999,48,5.46,7.13,6.63
2,2020-11-03,428719.425,nd,987253.6125,nd,160.892029,188,18.0,270,518,30.000000,49,5.53,7.28,6.65
3,2020-11-04,353631.014192,nd,725334.025614,nd,159.936325,196,18.0,280,542,29.299999,53,6.90,7.08,6.57
4,2020-11-05,511944.444444,nd,906111.111111,nd,158.947968,328,18.0,250,571,28.500000,51,5.79,7.22,6.29


In [10]:
df = df.replace("nd", np.nan)

In [11]:
for col in df.columns:
    print(col, df[col].dtype)

Date datetime64[ns]
N1 value (copies/L) float64
N1 Stnd error (copies/L) float64
N2 value (copies/L) float64
N2 Stnd error (copies/L) float64
Flow (million gallons/d) float64
Influent Suspended Solids (mg/l) int64
Influent Temperature (C) float64
Influent CBOD-5day Liquids (mg/l) int64
Influent COD (mg/l) int64
Influent Ammonia Nitrogen (mgN/l) float64
Influent Total Kjeldahl Nitrogen (mgTKN/l) int64
Influent Total Phosphorus (mgP/l float64
Influent Maximum pH (su) float64
Influent Minimum pH (su) float64


In [17]:
def gcl_2_gcml(vals):
    return vals * 0.001


def mgd_2_m3d(vals):
    return vals * 0.0037854118 * 1e6

variables = {
    "N1 value (copies/L)": {
        "table": "WWMeasure",
        "type": "covN1",
        "unit": "gcMl",
        "aggregation": "single",
        "qualityFlag": False    ,
        "transform": gcl_2_gcml,
    },
    "N1 Stnd error (copies/L)": {
        "table": "WWMeasure",
        "type": "covN1",
        "unit": "gcMl",
        "aggregation": "std",
        "qualityFlag": False,
        "transform": gcl_2_gcml,
    },
    "N2 value (copies/L)": {
        "table": "WWMeasure",
        "type": "covN2",
        "unit": "gcMl",
        "aggregation": "single",
        "qualityFlag": False,
        "transform": gcl_2_gcml,
    },
    "N2 Stnd error (copies/L)": {
        "table": "WWMeasure",
        "type": "covN2",
        "unit": "gcMl",
        "aggregation": "std",
        "qualityFlag": False,
        "transform": gcl_2_gcml,
    },
    "Flow (million gallons/d)": {
        "table": "SiteMeasure",
        "type": "wwFlow",
        "unit": "m3d",
        "aggregation": "mean",
        "qualityFlag": False,
        "transform": mgd_2_m3d,
    },
    "Influent Suspended Solids (mg/l)": {
        "table": "SiteMeasure",
        "type": "wwTSS",
        "unit": "mgL",
        "aggregation": "single",
        "qualityFlag": False,
        "transform": None,
    },
    "Influent Temperature (C)": {
        "table": "SiteMeasure",
        "type": "wwTemp",
        "unit": "degC",
        "aggregation": "mean",
        "qualityFlag": False,
        "transform": None,
    },
    "Influent CBOD-5day Liquids (mg/l)": {
        "table": "SiteMeasure",
        "type": "wwBOD5c",
        "unit": "mgL",
        "aggregation": "single",
        "qualityFlag": False,
        "transform": None,
    },
    "Influent COD (mg/l)": {
        "table": "SiteMeasure",
        "type": "wwCOD",
        "unit": "mgL",
        "aggregation": "single",
        "qualityFlag": False,
        "transform": None,
    },
    "Influent Ammonia Nitrogen (mgN/l)": {
        "table": "SiteMeasure",
        "type": "wwNH4N",
        "unit": "mgL",
        "aggregation": "single",
        "qualityFlag": False,
        "transform": None,
    },
    "Influent Total Kjeldahl Nitrogen (mgTKN/l)": {
        "table": "SiteMeasure",
        "type": "wwTKN",
        "unit": "mgL",
        "aggregation": "single",
        "qualityFlag": False,
        "transform": None,
    },
    "Influent Total Phosphorus (mgP/l": {
        "table": "SiteMeasure",
        "type": "wwPP",
        "unit": "mgL",
        "aggregation": "single",
        "qualityFlag": False,
        "transform": None,
    },
    "Influent Maximum pH (su)": {
        "table": "SiteMeasure",
        "type": "wwPh",
        "unit": "pH",
        "aggregation": "max",
        "qualityFlag": False,
        "transform": None,
    },
    "Influent Minimum pH (su)": {
        "table": "SiteMeasure",
        "type": "wwPh",
        "unit": "pH",
        "aggregation": "min",
        "qualityFlag": False,
        "transform": None,
    },
}

def build_prefix(info: dict) -> str:
    return "_".join([
        info["table"],
        info["type"],
        info["unit"],
        info["aggregation"],
        ])

df2 = df.copy()
for variable, info in variables.items():
    prefix = build_prefix(info)
    value_name = "_".join([prefix, "value"])
    qf_name = "_".join([prefix, "qualityFlag"])
    df2[variable] = info["transform"](df2[variable]) if info["transform"] is not None else df2[variable]
    df2 = df2.rename(columns={variable: value_name})
    df2[qf_name] = info["qualityFlag"]
df2 = df2.set_index("Date", drop=True)
df2.head()
val_cols = [col for col in df2.columns if "value" in str(col)]
df3 = df2[val_cols]
px.line(df3)

('Dakota', 'Washington', 'Ramsey', 'Anoka')